In [23]:
import brickschema
from rdflib import BNode
from brickschema import namespaces as ns

In [24]:
g = brickschema.GraphCollection()
g.load_graph("ttl/Brick.ttl")
g.load_graph("ttl/building.ttl", graph_name="building")

<Graph identifier=building (<class 'rdflib.graph.Graph'>)>

In [25]:
# fixing the graph

# update timeseries identifiers
for (point, ref, tid) in g.query("""SELECT ?point ?ref ?tid WHERE {
   ?point brick:timeseries ?ref .
   ?ref brick:hasTimeseriesId ?tid
}"""):
    bldg = g.graph("building")
    bldg.remove((point, ns.BRICK.timeseries, ref))
    bldg.remove((ref, ns.BRICK.hasTimeseriesId, tid))
    p = BNode()
    bldg.add((point, ns.REF.hasExternalReference, p))
    bldg.add((p, ns.A, ns.REF.TimeseriesReference))
    bldg.add((p, ns.REF.hasTimeseriesId, tid))
# g.serialize("ttl/x.ttl", format='turtle')
for ref in g.query("""SELECT ?ref WHERE {
   ?ref a brick:TimeseriesReference .
   FILTER NOT EXISTS { ?point ref:hasExternalReference ?ref }
}"""):
    triples = g.cbd(ref)
    for t in triples:
        g.remove(t)

for (point, loc) in g.query("""SELECT ?point ?loc WHERE {
?point rdf:type/rdfs:subClassOf* brick:Point .
?point brick:hasLocation ?loc .
?loc rdf:type/rdfs:subClassOf* brick:Location
}"""):
    bldg = g.graph("building")
    bldg.remove((point, ns.BRICK.hasLocation, loc))
    bldg.add((point, ns.BRICK.isPointOf, loc))

In [26]:
valid, _, report = g.validate()
assert valid, report

AssertionError: Validation Report
Conforms: False
Results (2):
Constraint Violation in ClassConstraintComponent (http://www.w3.org/ns/shacl#ClassConstraintComponent):
	Severity: sh:Violation
	Source Shape: [ sh:class brick:Location ; sh:message Literal("A Location's parts should be always Locations.") ; sh:path brick:hasPart ]
	Focus Node: <http://openmetrics.eu/openmetrics#Site_241>
	Value Node: <http://openmetrics.eu/openmetrics#Building_140>
	Result Path: brick:hasPart
	Message: A Location's parts should be always Locations.
Constraint Violation in ClassConstraintComponent (http://www.w3.org/ns/shacl#ClassConstraintComponent):
	Severity: sh:Violation
	Source Shape: [ sh:class brick:Location ; sh:message Literal("A Location's parts should be always Locations.") ; sh:path brick:hasPart ]
	Focus Node: <http://openmetrics.eu/openmetrics#Site_241>
	Value Node: <http://openmetrics.eu/openmetrics#Building_140>
	Result Path: brick:hasPart
	Message: A Location's parts should be always Locati

In [27]:
# run SHACL processor
g.expand("shacl")

<Graph identifier=N19bd8b9f31d44869aa39d05afc5c71cf (<class 'brickschema.graph.GraphCollection'>)>

In [28]:
g.graph("building").serialize("ttl/building-compiled.ttl")

<Graph identifier=building (<class 'rdflib.graph.Graph'>)>